In [1]:
import os
# Disable TensorFlow to avoid tf_keras import errors when using PyTorch
os.environ['TRANSFORMERS_NO_TF'] = '1'
os.environ['USE_TF'] = '0'

# Suppress warnings before importing transformers
import warnings 
warnings.filterwarnings('ignore')

import numpy as np    # linear algebra
import pandas as pd   # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch.utils.data import Dataset, DataLoader

# Import transformers (tf_keras is now installed to prevent import errors)
from transformers import (BertTokenizer, BertForSequenceClassification, BertModel, AdamWeightDecay, get_linear_schedule_with_warmup,
                          TrainingArguments, Trainer)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [31]:
import re

df_08 = pd.read_csv('S08_question_answer_pairs.txt', sep='\t')
df_09 = pd.read_csv('S09_question_answer_pairs.txt', sep='\t')
df_10 = pd.read_csv('S10_question_answer_pairs.txt', sep='\t', encoding='ISO-8859-1')

# 1) Basic cleaning
df = pd.concat([df_08, df_09, df_10])
df = df[['Question', 'Answer']].dropna()
df = df[(df['Question'].str.len() > 0) & (df['Answer'].str.len() > 0)]
df = df.drop_duplicates(subset=['Question'])

# 2) Normalize answers to avoid label explosion
#    - lowercase, strip
#    - remove trailing punctuation (. ! ?)
df['Answer'] = df['Answer'].astype(str).str.lower().str.strip()
df['Answer'] = df['Answer'].str.replace(r'[\"“”]', '', regex=True)
df['Answer'] = df['Answer'].str.replace(r'\s+', ' ', regex=True).str.strip()
df['Answer'] = df['Answer'].str.replace(r'[\.!\?]+$', '', regex=True).str.strip()



# 3) Filter rare answers AFTER normalization
min_count = 5  # raise/lower as needed; higher -> fewer classes
vc = df['Answer'].value_counts()
keep = vc[vc >= min_count].index
if len(keep) == 0:
    keep = vc.index  # fallback if threshold too high
    min_count = 1

df = df[df['Answer'].isin(keep)]

# 4) Keep top-K frequent answers; map others to "other"
top_k = 100  # adjust (e.g., 50 or 100)
vc = df['Answer'].value_counts()
keep = vc.index[:top_k]
df['Answer'] = df['Answer'].where(df['Answer'].isin(keep), other='other')


# 5) Re-fit LabelEncoder AFTER filtering
le = LabelEncoder()
df['Encoded_Answer'] = le.fit_transform(df['Answer'])
num_labels = len(le.classes_)
print(f"\nClasses after filtering: {num_labels}")
print(f"Top classes: {le.classes_[:10]}")

# 5) Stratified split (only if each class has >=2 samples)
min_class_count = df['Encoded_Answer'].value_counts().min()
use_stratify = (df['Encoded_Answer'].nunique() > 1) and (min_class_count > 1)

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Question'].tolist(),
    df['Encoded_Answer'].tolist(),
    test_size=0.2,
    random_state=42,
    stratify=df['Encoded_Answer'] if use_stratify else None
)

print(f"\nTrain size: {len(train_texts)}")
print(f"Val size: {len(val_texts)}")



Classes after filtering: 3
Top classes: ['no' 'three' 'yes']

Train size: 752
Val size: 188


In [41]:
# 3. Create Dataset Class for QA Classification
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 256  # Reduced from 512 for efficiency

class QADataset(Dataset):
    """
    Dataset class for Question-Answer classification
    Takes questions and labels (encoded answers) for sequence classification
    """
    def __init__(self, questions, labels, tokenizer, max_length=256):
        """
        Args:
            questions: List of question strings
            labels: List of integer labels (encoded answers)
            tokenizer: BERT tokenizer
            max_length: Maximum sequence length
        """
        self.questions = questions
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.questions)
    
    def __getitem__(self, idx):
        """
        Returns tokenized question with label
        Format compatible with Trainer API
        """
        question = str(self.questions[idx])
        label = int(self.labels[idx])
        
        # Tokenize question only (for classification task)
        encoding = self.tokenizer(
            question,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'token_type_ids': encoding['token_type_ids'].squeeze(0),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create datasets using the train/val split from Cell 1
print("="*70)
print("Creating Train and Validation Datasets")
print("="*70)

# Check if train_texts and val_texts exist from Cell 1
if 'train_texts' in globals() and 'val_texts' in globals():
    train_dataset = QADataset(train_texts, train_labels, tokenizer, max_length=max_length)
    val_dataset = QADataset(val_texts, val_labels, tokenizer, max_length=max_length)
    
    print(f"\nTrain dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(val_dataset)}")
    print(f"Number of classes: {len(le.classes_)}")
    
    # Test the dataset
    print("\n" + "="*70)
    print("Testing Dataset")
    print("="*70)
    
    sample = train_dataset[0]
    print(f"\nSample from training set:")
    print(f"  Input IDs shape: {sample['input_ids'].shape}")
    print(f"  Attention mask shape: {sample['attention_mask'].shape}")
    print(f"  Token type IDs shape: {sample['token_type_ids'].shape}")
    print(f"  Label: {sample['labels'].item()} (Answer: '{le.inverse_transform([sample['labels'].item()])[0]}')")
    print(f"  Original question: {train_texts[0][:100]}...")
    
    sample_val = val_dataset[0]
    print(f"\nSample from validation set:")
    print(f"  Label: {sample_val['labels'].item()} (Answer: '{le.inverse_transform([sample_val['labels'].item()])[0]}')")
    print(f"  Original question: {val_texts[0][:100]}...")
    
else:
    print("ERROR: train_texts and val_texts not found!")
    print("Please run Cell 1 first to create the train/val split.")
    print("\nCreating a simple example instead...")
    
    # Fallback: create a simple example
    sample_questions = df['Question'].head(10).tolist()
    sample_labels = df['Encoded_Answer'].head(10).tolist()
    example_dataset = QADataset(sample_questions, sample_labels, tokenizer, max_length=max_length)
    print(f"Example dataset size: {len(example_dataset)}")
    print(f"Sample: {example_dataset[0]}")

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x17e73e640>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: aabdbc44-e9c4-4ad3-a75e-cfc2658612dd)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /bert-base-uncased/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x17e73ec40>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: fa2e5600-ab73-4000-9aba-1891c5413308)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
Retrying in 2s [Retry 

Creating Train and Validation Datasets

Train dataset size: 752
Validation dataset size: 188
Number of classes: 3

Testing Dataset

Sample from training set:
  Input IDs shape: torch.Size([256])
  Attention mask shape: torch.Size([256])
  Token type IDs shape: torch.Size([256])
  Label: 2 (Answer: 'yes')
  Original question: Do sea otters live along the Pacific coast?...

Sample from validation set:
  Label: 2 (Answer: 'yes')
  Original question: Is a string first tuned to a standard pitch?...


In [58]:
# 4. Load BERT Model with LoRA (Low-Rank Adaptation) for Efficient Fine-Tuning

print("="*70)
print("Loading BERT Model with LoRA for Parameter-Efficient Fine-Tuning")
print("="*70)

# Check if PEFT is installed
try:
    import peft
    from peft import LoraConfig, get_peft_model, TaskType
    print(f"✓ PEFT version: {peft.__version__}")
except ImportError:
    print("❌ PEFT library not installed!")
    print("Installing PEFT...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "peft"])
    from peft import LoraConfig, get_peft_model, TaskType
    print("✓ PEFT installed successfully!")

from transformers import BertForSequenceClassification, BertConfig

# Get number of classes from label encoder
num_labels = len(le.classes_)
print(f"\nNumber of answer classes: {num_labels}")
print(f"Sample classes: {le.classes_[:10]}")

# Load base model (frozen, not trainable)
print("\n" + "="*70)
print("Step 1: Loading Base BERT Model")
print("="*70)

config = BertConfig.from_pretrained(
    'bert-base-uncased',
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    num_labels=num_labels
)

# Load pre-trained BERT model
base_model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    config=config
)

# Move to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
base_model = base_model.to(device)

print(f"\nBase model loaded on device: {device}")
print(f"Total parameters: {sum(p.numel() for p in base_model.parameters()):,}")

# Configure LoRA
print("\n" + "="*70)
print("Step 2: Configuring LoRA")
print("="*70)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence classification task
    r=8,                          # LoRA rank (rank of adaptation matrices) - lower = fewer params
    lora_alpha=16,               # LoRA alpha (scaling factor) - typically 2x rank
    lora_dropout=0.1,            # LoRA dropout
    target_modules=["query", "value", "key"],  # Which modules to apply LoRA to
    bias="none",                 # Don't train bias parameters
)

print(f"\nLoRA Configuration:")
print(f"  Rank (r): {lora_config.r}")
print(f"  Alpha: {lora_config.lora_alpha}")
print(f"  Dropout: {lora_config.lora_dropout}")
print(f"  Target Modules: {lora_config.target_modules}")
print(f"  Bias: {lora_config.bias}")

# Apply LoRA to the model
print("\n" + "="*70)
print("Step 3: Applying LoRA to Model")
print("="*70)

model = get_peft_model(base_model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

# Get parameter counts
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
frozen_params = total_params - trainable_params

print(f"\n" + "="*70)
print("Parameter Summary:")
print("="*70)
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,} ({trainable_params/total_params*100:.2f}%)")
print(f"  Frozen parameters: {frozen_params:,} ({frozen_params/total_params*100:.2f}%)")
print(f"\n✓ LoRA reduces trainable parameters by ~{(1-trainable_params/total_params)*100:.1f}%!")
print("  This means:")
print("  - Faster training")
print("  - Less memory usage")
print("  - Lower risk of overfitting")
print("="*70)


Loading BERT Model with LoRA for Parameter-Efficient Fine-Tuning
❌ PEFT library not installed!
Installing PEFT...
Defaulting to user installation because normal site-packages is not writeable
✓ PEFT installed successfully!

Number of answer classes: 3
Sample classes: ['no' 'three' 'yes']

Step 1: Loading Base BERT Model


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Base model loaded on device: cpu
Total parameters: 109,484,547

Step 2: Configuring LoRA

LoRA Configuration:
  Rank (r): 8
  Alpha: 16
  Dropout: 0.1
  Target Modules: {'key', 'query', 'value'}
  Bias: none

Step 3: Applying LoRA to Model
trainable params: 444,675 || all params: 109,929,222 || trainable%: 0.4045

Parameter Summary:
  Total parameters: 109,929,222
  Trainable parameters: 444,675 (0.40%)
  Frozen parameters: 109,484,547 (99.60%)

✓ LoRA reduces trainable parameters by ~99.6%!
  This means:
  - Faster training
  - Less memory usage
  - Lower risk of overfitting


## LoRA Configuration Guide

### What is LoRA?

**LoRA (Low-Rank Adaptation)** is a parameter-efficient fine-tuning technique that:
- Adds small trainable matrices to the model instead of training all parameters
- Reduces trainable parameters by 90-99%
- Maintains similar performance to full fine-tuning
- Uses less memory and trains faster

### LoRA Parameters Explained:

1. **`r` (rank)**: Rank of the low-rank matrices
   - Lower = fewer parameters (e.g., r=4, 8, 16)
   - Default: `r=8` (good balance)
   - Range: 4-64 (higher rarely needed)

2. **`lora_alpha`**: Scaling factor for LoRA weights
   - Typically set to 2x rank (e.g., r=8 → alpha=16)
   - Higher = stronger adaptation
   - Default: `16` (for r=8)

3. **`lora_dropout`**: Dropout for LoRA layers
   - Prevents overfitting
   - Default: `0.1` (can increase to 0.2 if overfitting)

4. **`target_modules`**: Which layers to apply LoRA to
   - `["query", "value", "key"]` - attention layers (recommended)
   - `["query", "value"]` - fewer parameters
   - `["dense"]` - feed-forward layers (less common)

### Adjusting LoRA Parameters:

**For more capacity (if underfitting):**
```python
r=16, lora_alpha=32  # More parameters
```

**For fewer parameters (if overfitting or memory constrained):**
```python
r=4, lora_alpha=8  # Fewer parameters
```

**For stronger regularization:**
```python
lora_dropout=0.2  # Higher dropout
```


## LoRA Benefits for Your QA Task

### Why Use LoRA?

1. **Reduced Memory Usage**
   - Only trains ~1-5% of parameters instead of 100%
   - Can train with larger batch sizes
   - Works on smaller GPUs/CPUs

2. **Faster Training**
   - Fewer parameters to update = faster iterations
   - Less computation per step

3. **Less Overfitting**
   - Smaller parameter space reduces overfitting risk
   - Especially useful for small datasets

4. **Multiple Task Adaptations**
   - Can train different LoRA adapters for different tasks
   - Keep base model, swap adapters

### Expected Results:

- **Trainable Parameters**: ~1-5% of full model (instead of 100%)
- **Performance**: Similar to full fine-tuning (often within 1-2%)
- **Training Speed**: 2-5x faster
- **Memory**: 50-80% reduction

### LoRA vs Full Fine-Tuning:

| Aspect | Full Fine-Tuning | LoRA |
|--------|------------------|------|
| Trainable Params | 100% (110M) | 1-5% (~1-5M) |
| Training Speed | Baseline | 2-5x faster |
| Memory Usage | High | Low |
| Overfitting Risk | Higher | Lower |
| Performance | Best | Very close (95-99%) |

### When to Use LoRA:

✅ **Use LoRA when:**
- Limited GPU memory
- Small dataset (high overfitting risk)
- Need to train multiple tasks
- Want faster experimentation

❌ **Use Full Fine-Tuning when:**
- Large dataset
- Need absolute best performance
- Have abundant compute resources


In [59]:
# 5. Setup Training Configuration

print("="*70)
print("Setting Up Training Configuration")
print("="*70)

# First, check if accelerate is installed
try:
    import accelerate
    print(f"✓ Accelerate version: {accelerate.__version__}")
except ImportError:
    print("❌ Accelerate not installed!")
    print("Run: pip install --upgrade accelerate")
    raise

from transformers import TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score
import numpy as np
import torch

def top_k_acc(logits, labels, k=3):
    topk = np.argsort(logits, axis=1)[:, ::-1][:, :k]
    return np.mean([labels[i] in topk[i] for i in range(len(labels))])

def compute_metrics(eval_pred):
    """
    Compute accuracy and F1 score for evaluation
    """
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    return {
        "accuracy": accuracy_score(labels, preds),
        "balanced_accuracy": balanced_accuracy_score(labels, preds),
        "micro_f1": f1_score(labels, preds, average="micro", zero_division=0),
        "weighted_f1": f1_score(labels, preds, average="weighted", zero_division=0),
        "macro_f1": f1_score(labels, preds, average="macro", zero_division=0),
        "top3_acc": top_k_acc(logits, labels, k=3),
    }
# Check available parameters
import inspect
training_args_params = inspect.signature(TrainingArguments.__init__).parameters
print(f"\nAvailable TrainingArguments parameters: {len(training_args_params)}")

# Build training arguments
# Increased learning rate to help model learn faster and break out of stuck state
base_args = {
    'output_dir': './bert_qa_sft_outputs',
    'num_train_epochs': 12,  # More epochs to allow learning
    'per_device_train_batch_size': 16,
    'per_device_eval_batch_size': 32,
    'learning_rate': 5e-5,  # Increased from 1e-5 to help model learn faster
    'weight_decay': 0.05,   # Slightly reduced to allow more learning
    'warmup_ratio': 0.1,
    'logging_dir': './logs',
    'logging_steps': 50,
    'save_total_limit': 2,
    'load_best_model_at_end': True,
    'metric_for_best_model': 'balanced_accuracy',  # Use balanced accuracy instead of loss
    'greater_is_better': True,  # Balanced accuracy: higher is better
}

# ✅ Set eval_strategy and save_strategy to match
if 'eval_strategy' in training_args_params:
    base_args['eval_strategy'] = 'epoch'
    base_args['save_strategy'] = 'epoch'
    print("✓ Using 'eval_strategy' (modern transformers)")
elif 'eval_strategy' in training_args_params:
    base_args['eval_strategy'] = 'epoch'
    base_args['save_strategy'] = 'epoch'
    print("✓ Using 'eval_strategy' (legacy transformers)")

# Add optional parameters
if 'dataloader_num_workers' in training_args_params:
    base_args['dataloader_num_workers'] = 0

if torch.cuda.is_available() and 'fp16' in training_args_params:
    base_args['fp16'] = True
    print("✓ FP16 enabled (GPU available)")

if 'report_to' in training_args_params:
    base_args['report_to'] = 'none'

# Create TrainingArguments
try:
    training_args = TrainingArguments(**base_args)
    print("\n✓ TrainingArguments created successfully!")
except Exception as e:
    print(f"\n❌ Error creating TrainingArguments:")
    print(f"   {str(e)}")
    raise

# Print configuration
print("\n" + "="*70)
print("Training Configuration Summary")
print("="*70)
print(f"  Learning Rate: {training_args.learning_rate}")
print(f"  Batch Size: {training_args.per_device_train_batch_size}")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Weight Decay: {training_args.weight_decay}")
print(f"  Warmup Ratio: {training_args.warmup_ratio}")
print(f"  Device: {training_args.device}")
print(f"  FP16: {getattr(training_args, 'fp16', False)}")
print(f"  Eval Strategy: {getattr(training_args, 'eval_strategy', getattr(training_args, 'evaluation_strategy', 'N/A'))}")
print(f"  Save Strategy: {getattr(training_args, 'save_strategy', 'N/A')}")
print(f"  Load Best Model: {training_args.load_best_model_at_end}")
print("="*70)

print("\n✓ Training configuration ready!")

Setting Up Training Configuration
✓ Accelerate version: 1.10.1

Available TrainingArguments parameters: 133
✓ Using 'eval_strategy' (modern transformers)

✓ TrainingArguments created successfully!

Training Configuration Summary
  Learning Rate: 5e-05
  Batch Size: 16
  Epochs: 12
  Weight Decay: 0.05
  Warmup Ratio: 0.1
  Device: mps
  FP16: False
  Eval Strategy: epoch
  Save Strategy: epoch
  Load Best Model: True

✓ Training configuration ready!


In [60]:
# 5.5. Analyze Class Distribution and Create Weighted Loss Trainer

print("="*70)
print("Analyzing Class Distribution and Creating Weighted Loss")
print("="*70)

from collections import Counter
import torch.nn as nn

# Analyze class distribution in training set
train_label_counts = Counter(train_labels)
print("\nClass Distribution in Training Set:")
for label_id, count in sorted(train_label_counts.items()):
    class_name = le.inverse_transform([label_id])[0]
    print(f"  Class {label_id} ('{class_name}'): {count} samples")

# Calculate class weights (inverse frequency with stronger weighting)
total_samples = len(train_labels)
class_weights = []
max_count = max(train_label_counts.values()) if train_label_counts else 1

print(f"\nCalculating class weights (total samples: {total_samples}):")
for i in range(num_labels):
    count = train_label_counts.get(i, 1)  # Avoid division by zero
    # Stronger inverse frequency weighting (square root for less extreme weights)
    # This gives minority classes much higher weight
    weight = np.sqrt(max_count / count) if count > 0 else 1.0
    class_weights.append(weight)
    class_name = le.inverse_transform([i])[0]
    print(f"  Class {i} ('{class_name}'): {count} samples → weight: {weight:.3f}")

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)
print(f"\nClass Weights Tensor: {class_weights_tensor.tolist()}")
print(f"Weight ratio (max/min): {max(class_weights) / min(class_weights):.2f}x")

# Create custom Trainer with weighted loss
from transformers import Trainer

class WeightedTrainer(Trainer):
    """
    Custom Trainer with class-weighted loss to handle imbalanced classes
    """
    def __init__(self, class_weights, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weights = class_weights.to(self.model.device)  # Ensure weights on same device
        print(f"\n✓ WeightedTrainer initialized with class weights on device: {self.class_weights.device}")
    
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        Override compute_loss to use weighted cross-entropy
        Accepts **kwargs for compatibility with newer transformers versions
        """
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # Ensure class weights are on the same device as logits
        weights = self.class_weights.to(logits.device)
        
        # Use weighted cross-entropy loss
        loss_fct = nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss
    
    def log(self, logs, start_time=None):
        """
        Override log to show class prediction distribution
        Matches parent class signature for compatibility
        """
        # Call parent log with correct signature
        if start_time is not None:
            super().log(logs, start_time)
        else:
            super().log(logs)
        
        # Every 100 steps, log prediction distribution (optional debugging)
        if hasattr(self.state, 'global_step') and self.state.global_step % 100 == 0 and 'loss' in logs:
            try:
                self.model.eval()
                sample_batch = next(iter(self.get_train_dataloader()))
                with torch.no_grad():
                    # Move batch to device
                    batch_inputs = {k: v.to(self.model.device) for k, v in sample_batch.items() if k != 'labels'}
                    outputs = self.model(**batch_inputs)
                    preds = torch.argmax(outputs.logits, dim=-1).cpu().numpy()
                    pred_dist = Counter(preds)
                    if len(pred_dist) > 1:
                        print(f"  Step {self.state.global_step}: Prediction diversity - {len(pred_dist)} classes predicted")
                self.model.train()
            except Exception as e:
                pass  # Skip if there's an error (non-critical)

print("\n✓ Weighted Trainer class created!")
print("  This will help the model learn from minority classes better")


Analyzing Class Distribution and Creating Weighted Loss

Class Distribution in Training Set:
  Class 0 ('no'): 138 samples
  Class 1 ('three'): 4 samples
  Class 2 ('yes'): 610 samples

Calculating class weights (total samples: 752):
  Class 0 ('no'): 138 samples → weight: 2.102
  Class 1 ('three'): 4 samples → weight: 12.349
  Class 2 ('yes'): 610 samples → weight: 1.000

Class Weights Tensor: [2.1024484634399414, 12.349088668823242, 1.0]
Weight ratio (max/min): 12.35x

✓ Weighted Trainer class created!
  This will help the model learn from minority classes better


In [61]:
# 6.1. Save LoRA Model (After Training)

print("="*70)
print("Saving LoRA Model")
print("="*70)

final_model_path = './bert_qa_sft_final_model'

# Save LoRA adapters (if using LoRA)
try:
    from peft import PeftModel
    if isinstance(model, PeftModel) or hasattr(model, 'peft_config'):
        # Save LoRA adapters
        model.save_pretrained(final_model_path)
        tokenizer.save_pretrained(final_model_path)
        
        print(f"\n✓ LoRA adapters saved to: {final_model_path}")
        print("  (Only LoRA weights are saved, base model is not included)")
        print("\n  To use this model later:")
        print("  ```python")
        print("  from transformers import BertForSequenceClassification")
        print("  from peft import PeftModel")
        print("  ")
        print("  # Load base model")
        print("  base_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)")
        print("  ")
        print("  # Load LoRA adapters")
        print(f"  model = PeftModel.from_pretrained(base_model, '{final_model_path}')")
        print("  ```")
        
        # Optional: Merge and save full model (uncomment if you want a standalone model)
        print("\n" + "="*70)
        print("Optional: Merge LoRA with Base Model")
        print("="*70)
        print("Uncomment the code below to merge LoRA adapters with base model:")
        print("  # merged_model = model.merge_and_unload()")
        print("  # merged_model.save_pretrained(final_model_path + '_merged')")
        print("  # tokenizer.save_pretrained(final_model_path + '_merged')")
    else:
        # Regular model saving (if not using LoRA)
        model.save_pretrained(final_model_path)
        tokenizer.save_pretrained(final_model_path)
        print(f"\n✓ Full model saved to: {final_model_path}")
except Exception as e:
    print(f"⚠ Error saving model: {e}")
    print("Trying standard save method...")
    model.save_pretrained(final_model_path)
    tokenizer.save_pretrained(final_model_path)
    print(f"✓ Model saved to: {final_model_path}")


Saving LoRA Model

✓ LoRA adapters saved to: ./bert_qa_sft_final_model
  (Only LoRA weights are saved, base model is not included)

  To use this model later:
  ```python
  from transformers import BertForSequenceClassification
  from peft import PeftModel
  
  # Load base model
  base_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)
  
  # Load LoRA adapters
  model = PeftModel.from_pretrained(base_model, './bert_qa_sft_final_model')
  ```

Optional: Merge LoRA with Base Model
Uncomment the code below to merge LoRA adapters with base model:
  # merged_model = model.merge_and_unload()
  # merged_model.save_pretrained(final_model_path + '_merged')
  # tokenizer.save_pretrained(final_model_path + '_merged')


In [62]:
# 6. Initialize Weighted Trainer and Start Training

print("="*70)
print("Initializing Weighted Trainer and Starting Training")
print("="*70)

# Use WeightedTrainer if available, otherwise fall back to regular Trainer
try:
    # Check if WeightedTrainer was created in previous cell
    if 'WeightedTrainer' in globals() and 'class_weights_tensor' in globals():
        print("✓ Using WeightedTrainer with class weights")
        trainer = WeightedTrainer(
            class_weights=class_weights_tensor,
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(
                early_stopping_patience=3,
                early_stopping_threshold=0.0
            )]
        )
    else:
        # Fallback to regular Trainer
        print("⚠ WeightedTrainer not found, using regular Trainer")
        from transformers import Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(
                early_stopping_patience=3,
                early_stopping_threshold=0.0
            )]
        )
except Exception as e:
    print(f"⚠ Error creating WeightedTrainer: {e}")
    print("Falling back to regular Trainer...")
    from transformers import Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(
            early_stopping_patience=3,
            early_stopping_threshold=0.0
        )]
    )

print("\n✓ Trainer initialized!")
print("\nStarting training...")
print("="*70)

# Train the model
trainer.train()

print("\n" + "="*70)
print("Training Completed!")
print("="*70)

# Save the final model
final_model_path = './bert_qa_sft_final_model'
trainer.save_model(final_model_path)
tokenizer.save_pretrained(final_model_path)
print(f"\n✓ Model saved to: {final_model_path}")


Initializing Weighted Trainer and Starting Training
✓ Using WeightedTrainer with class weights

✓ WeightedTrainer initialized with class weights on device: mps:0

✓ Trainer initialized!

Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,Balanced Accuracy,Micro F1,Weighted F1,Macro F1,Top3 Acc
1,No log,0.957013,0.813830,0.333333,0.813830,0.730299,0.299120,1.000000
2,1.102500,0.769136,0.813830,0.333333,0.813830,0.730299,0.299120,1.000000
3,0.809000,0.762343,0.813830,0.333333,0.813830,0.730299,0.299120,1.000000
4,0.775300,0.765116,0.813830,0.333333,0.813830,0.730299,0.299120,1.000000



Training Completed!

✓ Model saved to: ./bert_qa_sft_final_model


In [57]:
# 6.5. Diagnostic: Check Model Predictions Before Training

print("="*70)
print("Diagnostic: Checking Model Predictions (Before Training)")
print("="*70)

# Ensure model is on correct device
model = model.to(device)
model.eval()

# Check device
print(f"Model device: {next(model.parameters()).device}")
print(f"Target device: {device}")

# Get a few predictions from untrained model
sample_indices = [0, 1, 2, 5, 10]
predictions_before = []

try:
    with torch.no_grad():
        for idx in sample_indices:
            if idx >= len(val_dataset):
                continue
                
            sample = val_dataset[idx]
            
            # Ensure all tensors are on the same device
            input_ids = sample['input_ids'].unsqueeze(0).to(device)
            attention_mask = sample['attention_mask'].unsqueeze(0).to(device)
            token_type_ids = sample['token_type_ids'].unsqueeze(0).to(device)
            
            # Make prediction
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )
            
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=-1)
            pred_id = torch.argmax(logits, dim=1).item()
            true_id = sample['labels'].item()
            
            predictions_before.append({
                'question': val_texts[idx][:60] + "..." if idx < len(val_texts) else "N/A",
                'true_label': true_id,
                'true_answer': le.inverse_transform([true_id])[0],
                'pred_label': pred_id,
                'pred_answer': le.inverse_transform([pred_id])[0],
                'confidence': probs[0][pred_id].item()
            })
    
    print("\nSample Predictions (Before Training):")
    for i, pred in enumerate(predictions_before, 1):
        match = "✓" if pred['true_label'] == pred['pred_label'] else "✗"
        print(f"\n{i}. {match} Question: {pred['question']}")
        print(f"   True: {pred['true_answer']} | Predicted: {pred['pred_answer']} (conf: {pred['confidence']:.3f})")
    
    # Check if model is predicting the same class for all samples
    if predictions_before:
        all_preds = [p['pred_label'] for p in predictions_before]
        unique_preds = len(set(all_preds))
        print(f"\n📊 Prediction Diversity: {unique_preds} unique predictions out of {len(predictions_before)} samples")
        if unique_preds == 1:
            print("⚠ WARNING: Model is predicting the same class for all samples!")
            print("   This suggests severe class imbalance or initialization issue.")
    
    print("\n✓ Diagnostic complete!")
    
except Exception as e:
    print(f"\n⚠ Error during diagnostic: {e}")
    print("This is okay - you can skip this diagnostic and proceed to training.")
    print("The weighted loss trainer should still work correctly.")


Diagnostic: Checking Model Predictions (Before Training)
Model device: cpu
Target device: cpu

Sample Predictions (Before Training):

1. ✓ Question: Is a string first tuned to a standard pitch?...
   True: yes | Predicted: yes (conf: 0.912)

2. ✓ Question: Did Henri Becquerel intentionally discover radioactivity?...
   True: no | Predicted: no (conf: 0.559)

3. ✓ Question: Was Tesla rich at the time of his death?...
   True: no | Predicted: no (conf: 0.748)

4. ✓ Question: Do his works enjoy a broad popular appeal in the United Stat...
   True: yes | Predicted: yes (conf: 0.955)

5. ✓ Question: Was it derived from Latin?...
   True: yes | Predicted: yes (conf: 0.951)

📊 Prediction Diversity: 2 unique predictions out of 5 samples

✓ Diagnostic complete!


# 9. Comprehensive Parameter Tuning Guide for 99.9% Accuracy

## Critical Problem Identified:

Your class distribution shows:
- **Class 0 ('no')**: 138 samples (18%)
- **Class 1 ('three')**: 4 samples (0.5%) ⚠️ **TOO FEW TO LEARN**
- **Class 2 ('yes')**: 610 samples (81%) ⚠️ **DOMINANT CLASS**

**This extreme imbalance makes 99.9% accuracy very difficult!**

## Why 99.9% is Challenging:

1. **Class 1 has only 4 samples** - Model cannot learn this class reliably
2. **81% of data is 'yes'** - Model defaults to predicting 'yes'
3. **Metrics stuck** - Model not learning, just predicting majority class

## Solutions (In Order of Priority):


## Realistic Expectations for 99.9% Accuracy

### Current Situation:

- **Class 1 ('three') has only 4 samples** - This is the main blocker
- **81% of data is 'yes'** - Severe imbalance
- **Metrics stuck** - Model not learning

### Is 99.9% Realistic?

**For your current dataset: NO** - Here's why:

1. **Class 1 is too rare** (4 samples) - Cannot learn reliably
2. **Class imbalance too extreme** (81% vs 0.5%)
3. **Small dataset** (~750 training samples)

### What You Can Achieve:

| Scenario | Realistic Accuracy |
|----------|-------------------|
| **Remove Class 1** (binary: yes/no) | **95-98%** ✅ |
| **Keep all 3 classes, balanced** | **85-92%** |
| **Keep all 3 classes, current imbalance** | **80-85%** (current) |
| **99.9% with current data** | **Not realistic** ❌ |

### Recommended Path to High Accuracy:

1. **Remove or merge Class 1** → Binary classification (yes/no)
2. **Balance remaining classes** → Oversample 'no' to match 'yes'
3. **Use optimized hyperparameters** → Higher LR, more epochs
4. **Full fine-tuning** → Not LoRA (for maximum performance)
5. **Ensemble models** → Train 3-5 models, average predictions

### Expected Results After Fixes:

- **Binary (yes/no)**: 95-98% accuracy ✅
- **3-class balanced**: 85-92% accuracy
- **99.9%**: Requires much more data or different approach


## What Was Wrong With Your Training

### Root Causes Identified:

1. **Model Stuck Predicting "yes" Class**
   - Diagnostic showed model predicts "yes" for ALL samples
   - This means the model never learned to differentiate classes
   - "yes" is likely the most frequent class (80%+ of data)

2. **Learning Rate Too Low**
   - Previous: `1e-5` was too conservative
   - Model couldn't break out of initial bias
   - **Fixed**: Increased to `3e-5` for faster learning

3. **Class Weights Not Strong Enough**
   - Previous formula: `total_samples / (num_labels * count)`
   - Didn't give enough penalty to majority class
   - **Fixed**: Using `sqrt(max_count / count)` for stronger weighting

4. **Wrong Metric for Best Model**
   - Previous: Used `eval_loss` (model optimized for frequent class)
   - **Fixed**: Now uses `balanced_accuracy` (optimizes for all classes)

5. **Metrics Not Changing**
   - All metrics identical = model not learning
   - Need stronger class weights + higher learning rate

### Improvements Made:

✅ **Stronger Class Weights**: `sqrt(max_count / count)` gives minority classes more weight  
✅ **Higher Learning Rate**: `3e-5` (was `1e-5`) to break out of stuck state  
✅ **Better Metric**: `balanced_accuracy` instead of `eval_loss`  
✅ **More Epochs**: 8 epochs (was 6) to allow more learning  
✅ **Device Safety**: Better handling of class weights on correct device  
✅ **Prediction Tracking**: Logs prediction diversity during training  

### What to Expect Now:

- **Balanced Accuracy**: Should increase from 0.33 to 0.5+ (ideally 0.7+)
- **Macro F1**: Should increase from 0.30 to 0.5+ (ideally 0.7+)
- **Metrics Will Change**: No longer identical across epochs
- **Prediction Diversity**: Model should predict different classes, not just "yes"

### Next Steps:

1. **Re-run Cell 4** (training config) - to get new learning rate
2. **Re-run Cell 5.5** (weighted trainer) - to get stronger class weights
3. **Re-run Cell 6** (training) - should see improvement now

If still stuck after these changes:
- Further increase learning rate to `5e-5`
- Reduce dropout to `0.1` (may be too high at 0.25)
- Check if class distribution is too extreme (one class >90% of data)


In [22]:
# 7. Evaluate Model Performance

print("="*70)
print("Evaluating Model Performance")
print("="*70)

# Evaluate on validation set
eval_results = trainer.evaluate()

print("\nValidation Set Results:")
print(f"  Loss: {eval_results['eval_loss']:.4f}")
print(f"  Accuracy: {eval_results['eval_accuracy']:.4f} ({eval_results['eval_accuracy']*100:.2f}%)")
print(f"  F1 Score: {eval_results['eval_f1']:.4f}")

# Get predictions for detailed analysis
print("\n" + "="*70)
print("Getting Detailed Predictions")
print("="*70)

predictions = trainer.predict(val_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

# Calculate per-class accuracy
from sklearn.metrics import classification_report, confusion_matrix

print("\nClassification Report:")
print(classification_report(
    true_labels, 
    predicted_labels, 
    target_names=[str(cls) for cls in le.classes_[:20]],  # Show first 20 classes
    zero_division=0
))

print("\n✓ Evaluation completed!")


Evaluating Model Performance



Validation Set Results:
  Loss: 0.4816
  Accuracy: 0.8138 (81.38%)
  F1 Score: 0.2991

Getting Detailed Predictions

Classification Report:
              precision    recall  f1-score   support

          no       0.00      0.00      0.00        34
       three       0.00      0.00      0.00         1
         yes       0.81      1.00      0.90       153

    accuracy                           0.81       188
   macro avg       0.27      0.33      0.30       188
weighted avg       0.66      0.81      0.73       188


✓ Evaluation completed!


## Summary: Improvements to Fix Low Accuracy

### Key Problems Identified:

1. **Model Stuck Predicting Same Class**: All metrics identical across epochs suggests model always predicts the most frequent class
2. **Severe Class Imbalance**: Balanced accuracy (0.33) vs Accuracy (0.81) shows model ignores minority classes
3. **No Class Weighting**: Loss function treats all classes equally, so model optimizes for frequent classes

### Solutions Implemented:

1. **✅ Class-Weighted Loss Function** (Cell 5.5)
   - Created `WeightedTrainer` that uses inverse frequency weighting
   - Minority classes get higher weights in loss calculation
   - Forces model to learn from all classes, not just frequent ones

2. **✅ Class Distribution Analysis** (Cell 5.5)
   - Shows exact distribution of classes in training set
   - Calculates appropriate weights for each class
   - Helps identify if certain classes are too rare

3. **✅ Diagnostic Tools** (Cell 6.5)
   - Checks model predictions before training
   - Identifies if model is stuck predicting same class
   - Helps debug initialization issues

### Expected Improvements:

- **Balanced Accuracy**: Should increase from 0.33 to 0.5+ (ideally 0.7+)
- **Macro F1**: Should increase from 0.30 to 0.5+ (ideally 0.7+)
- **Accuracy**: May decrease slightly (from 0.81) but overall performance improves
- **Metrics Should Change**: No longer identical across epochs

### Additional Recommendations:

1. **If still stuck**: 
   - Increase learning rate to 2e-5 or 3e-5
   - Reduce dropout to 0.1 (may be too high)
   - Check if class weights are too extreme

2. **If balanced accuracy improves but still low**:
   - Further reduce number of classes (top-K = 50 instead of 100)
   - Increase min_count threshold
   - Consider oversampling minority classes

3. **Monitor training**:
   - Watch for balanced_accuracy improving
   - Check if predictions become more diverse
   - Ensure loss is decreasing


In [ ]:
# 8. Inference Function - Predict Answers for New Questions

print("="*70)
print("Creating Inference Function")
print("="*70)

def predict_answer(question, model, tokenizer, label_encoder, device='cpu', top_k=top_k):
    """
    Predict answer for a given question
    
    Args:
        question: Question string
        model: Trained BERT model
        tokenizer: BERT tokenizer
        label_encoder: LabelEncoder used for encoding answers
        device: Device to run inference on
        top_k: Number of top predictions to return
    
    Returns:
        Dictionary with predicted answer and confidence scores
    """
    model.eval()
    
    # Tokenize question
    encoding = tokenizer(
        question,
        max_length=256,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    # Move to device
    encoding = {k: v.to(device) for k, v in encoding.items()}
    
    # Get predictions
    with torch.no_grad():
        outputs = model(**encoding)
        logits = outputs.logits
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
    
    # Get top-k predictions
    top_probs, top_indices = torch.topk(probabilities[0], k=min(top_k, len(label_encoder.classes_)))
    
    # Decode predictions
    results = []
    for prob, idx in zip(top_probs, top_indices):
        answer = label_encoder.inverse_transform([idx.item()])[0]
        results.append({
            'answer': answer,
            'confidence': prob.item(),
            'confidence_pct': f"{prob.item()*100:.2f}%"
        })
    
    return {
        'question': question,
        'predicted_answer': results[0]['answer'],
        'confidence': results[0]['confidence'],
        'top_predictions': results
    }

# Test the inference function
print("\nTesting Inference Function:")
print("="*70)

test_questions = [
    "Was Abraham Lincoln the sixteenth President of the United States?",
    "Did Lincoln sign the National Banking Act of 1863?",
    "Did his mother die of pneumonia?",
]

for question in test_questions:
    result = predict_answer(question, model, tokenizer, le, device=device)
    print(f"\nQuestion: {question}")
    print(f"Predicted Answer: {result['predicted_answer']}")
    print(f"Confidence: {result['confidence_pct']}")
    if len(result['top_predictions']) > 1:
        print(f"Top 3 Predictions:")
        for i, pred in enumerate(result['top_predictions'][:3], 1):
            print(f"  {i}. {pred['answer']} ({pred['confidence_pct']})")

print("\n" + "="*70)
print("✓ Inference function ready!")
print("="*70)


Creating Inference Function

Testing Inference Function:

Question: Was Abraham Lincoln the sixteenth President of the United States?
Predicted Answer: The Sud-Ouest borough was home to much of the city's industry during the late 19th and early-to-mid 20th century. 


KeyError: 'confidence_pct'

## Summary: Super Fine-Tuning BERT for QA Tasks

### What We Did:

1. **Data Preparation**
   - Loaded and cleaned question-answer pairs
   - Encoded answers as labels for classification
   - Split data into train/validation sets (80/20)

2. **Dataset Creation**
   - Created `QADataset` class compatible with HuggingFace Trainer
   - Tokenized questions with BERT tokenizer
   - Prepared data in format: `{input_ids, attention_mask, token_type_ids, labels}`

3. **Model Setup**
   - Loaded `bert-base-uncased` with classification head
   - Configured for multi-class classification (number of answer classes)

4. **Training Configuration**
   - Learning rate: 2e-5 (conservative for fine-tuning)
   - Weight decay: 0.05 (regularization)
   - Batch size: 16 (training), 32 (evaluation)
   - Early stopping: patience=2 epochs
   - Mixed precision (FP16) if GPU available

5. **Training & Evaluation**
   - Trained with automatic evaluation after each epoch
   - Saved best model based on validation loss
   - Computed accuracy and F1 scores

6. **Inference**
   - Created function to predict answers for new questions
   - Returns top-k predictions with confidence scores

### Key Features:

- ✅ **Super Fine-Tuning**: Conservative learning rate (2e-5) for stable training
- ✅ **Early Stopping**: Prevents overfitting
- ✅ **Best Model Saving**: Automatically saves best checkpoint
- ✅ **Evaluation Metrics**: Accuracy and F1 score
- ✅ **Inference Ready**: Easy-to-use prediction function

### Next Steps to Improve:

1. **If validation loss is high**:
   - Reduce learning rate to 1e-5
   - Increase weight decay to 0.1
   - Freeze lower BERT layers (uncomment in Cell 4)

2. **If overfitting**:
   - Increase dropout in model config
   - Use more aggressive early stopping (patience=1)
   - Reduce number of epochs

3. **If underfitting**:
   - Increase learning rate to 3e-5
   - Train for more epochs
   - Reduce weight decay

4. **For better performance**:
   - Use larger batch size if GPU memory allows
   - Try different learning rate schedules (cosine annealing)
   - Fine-tune hyperparameters based on validation results


In [63]:
# 10. Comparison: Base Model (No SFT) vs Fine-Tuned Model (SFT)

print("="*70)
print("Comparison: Base Model vs Fine-Tuned Model")
print("="*70)

import torch
from transformers import BertForSequenceClassification, BertConfig
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score, classification_report, confusion_matrix
import numpy as np

# Step 1: Load base model (no fine-tuning)
print("\n" + "="*70)
print("Step 1: Loading Base BERT Model (No Fine-Tuning)")
print("="*70)

base_model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=num_labels
)
base_model = base_model.to(device)
base_model.eval()

print(f"✓ Base model loaded on device: {device}")

# Step 2: Evaluate base model on validation set
print("\n" + "="*70)
print("Step 2: Evaluating Base Model")
print("="*70)

base_predictions = []
base_true_labels = []
base_logits_all = []

with torch.no_grad():
    for i in range(len(val_dataset)):
        sample = val_dataset[i]
        input_ids = sample['input_ids'].unsqueeze(0).to(device)
        attention_mask = sample['attention_mask'].unsqueeze(0).to(device)
        token_type_ids = sample['token_type_ids'].unsqueeze(0).to(device)
        
        outputs = base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        logits = outputs.logits.cpu().numpy()[0]
        pred = np.argmax(logits)
        true_label = sample['labels'].item()
        
        base_predictions.append(pred)
        base_true_labels.append(true_label)
        base_logits_all.append(logits)

base_predictions = np.array(base_predictions)
base_true_labels = np.array(base_true_labels)
base_logits_all = np.array(base_logits_all)

# Calculate metrics for base model
base_accuracy = accuracy_score(base_true_labels, base_predictions)
base_balanced_accuracy = balanced_accuracy_score(base_true_labels, base_predictions)
base_micro_f1 = f1_score(base_true_labels, base_predictions, average='micro', zero_division=0)
base_weighted_f1 = f1_score(base_true_labels, base_predictions, average='weighted', zero_division=0)
base_macro_f1 = f1_score(base_true_labels, base_predictions, average='macro', zero_division=0)

# Calculate loss (cross-entropy)
base_loss = torch.nn.functional.cross_entropy(
    torch.tensor(base_logits_all),
    torch.tensor(base_true_labels)
).item()

print(f"\nBase Model Results:")
print(f"  Loss: {base_loss:.4f}")
print(f"  Accuracy: {base_accuracy:.4f} ({base_accuracy*100:.2f}%)")
print(f"  Balanced Accuracy: {base_balanced_accuracy:.4f} ({base_balanced_accuracy*100:.2f}%)")
print(f"  Micro F1: {base_micro_f1:.4f}")
print(f"  Weighted F1: {base_weighted_f1:.4f}")
print(f"  Macro F1: {base_macro_f1:.4f}")

# Step 3: Get fine-tuned model results
print("\n" + "="*70)
print("Step 3: Getting Fine-Tuned Model Results")
print("="*70)

# Use the trainer to evaluate fine-tuned model
try:
    if 'trainer' in globals():
        sft_results = trainer.evaluate()
        sft_loss = sft_results.get('eval_loss', 0)
        sft_accuracy = sft_results.get('eval_accuracy', 0)
        sft_balanced_accuracy = sft_results.get('eval_balanced_accuracy', 0)
        sft_micro_f1 = sft_results.get('eval_micro_f1', 0)
        sft_weighted_f1 = sft_results.get('eval_weighted_f1', 0)
        sft_macro_f1 = sft_results.get('eval_macro_f1', 0)
    elif 'trainer_optimized' in globals():
        sft_results = trainer_optimized.evaluate()
        sft_loss = sft_results.get('eval_loss', 0)
        sft_accuracy = sft_results.get('eval_accuracy', 0)
        sft_balanced_accuracy = sft_results.get('eval_balanced_accuracy', 0)
        sft_micro_f1 = sft_results.get('eval_micro_f1', 0)
        sft_weighted_f1 = sft_results.get('eval_weighted_f1', 0)
        sft_macro_f1 = sft_results.get('eval_macro_f1', 0)
    else:
        # Evaluate fine-tuned model manually
        model.eval()
        sft_predictions = []
        sft_true_labels = []
        sft_logits_all = []
        
        with torch.no_grad():
            for i in range(len(val_dataset)):
                sample = val_dataset[i]
                input_ids = sample['input_ids'].unsqueeze(0).to(device)
                attention_mask = sample['attention_mask'].unsqueeze(0).to(device)
                token_type_ids = sample['token_type_ids'].unsqueeze(0).to(device)
                
                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids
                )
                
                logits = outputs.logits.cpu().numpy()[0]
                pred = np.argmax(logits)
                true_label = sample['labels'].item()
                
                sft_predictions.append(pred)
                sft_true_labels.append(true_label)
                sft_logits_all.append(logits)
        
        sft_predictions = np.array(sft_predictions)
        sft_true_labels = np.array(sft_true_labels)
        sft_logits_all = np.array(sft_logits_all)
        
        sft_loss = torch.nn.functional.cross_entropy(
            torch.tensor(sft_logits_all),
            torch.tensor(sft_true_labels)
        ).item()
        sft_accuracy = accuracy_score(sft_true_labels, sft_predictions)
        sft_balanced_accuracy = balanced_accuracy_score(sft_true_labels, sft_predictions)
        sft_micro_f1 = f1_score(sft_true_labels, sft_predictions, average='micro', zero_division=0)
        sft_weighted_f1 = f1_score(sft_true_labels, sft_predictions, average='weighted', zero_division=0)
        sft_macro_f1 = f1_score(sft_true_labels, sft_predictions, average='macro', zero_division=0)
    
    print(f"\nFine-Tuned Model Results:")
    print(f"  Loss: {sft_loss:.4f}")
    print(f"  Accuracy: {sft_accuracy:.4f} ({sft_accuracy*100:.2f}%)")
    print(f"  Balanced Accuracy: {sft_balanced_accuracy:.4f} ({sft_balanced_accuracy*100:.2f}%)")
    print(f"  Micro F1: {sft_micro_f1:.4f}")
    print(f"  Weighted F1: {sft_weighted_f1:.4f}")
    print(f"  Macro F1: {sft_macro_f1:.4f}")
    
except Exception as e:
    print(f"⚠ Error getting fine-tuned results: {e}")
    print("Using values from training history...")
    # Use the latest training metrics
    sft_loss = 0.765116  # From epoch 4
    sft_accuracy = 0.813830
    sft_balanced_accuracy = 0.333333
    sft_micro_f1 = 0.813830
    sft_weighted_f1 = 0.730299
    sft_macro_f1 = 0.299120

# Step 4: Create comparison table
print("\n" + "="*70)
print("Step 4: Side-by-Side Comparison")
print("="*70)

comparison_data = {
    'Metric': ['Loss (Lower is Better)', 'Accuracy', 'Balanced Accuracy', 'Micro F1', 'Weighted F1', 'Macro F1'],
    'Base Model (No SFT)': [
        f"{base_loss:.4f}",
        f"{base_accuracy:.4f} ({base_accuracy*100:.2f}%)",
        f"{base_balanced_accuracy:.4f} ({base_balanced_accuracy*100:.2f}%)",
        f"{base_micro_f1:.4f}",
        f"{base_weighted_f1:.4f}",
        f"{base_macro_f1:.4f}"
    ],
    'Fine-Tuned Model (SFT)': [
        f"{sft_loss:.4f}",
        f"{sft_accuracy:.4f} ({sft_accuracy*100:.2f}%)",
        f"{sft_balanced_accuracy:.4f} ({sft_balanced_accuracy*100:.2f}%)",
        f"{sft_micro_f1:.4f}",
        f"{sft_weighted_f1:.4f}",
        f"{sft_macro_f1:.4f}"
    ]
}

import pandas as pd
comparison_df = pd.DataFrame(comparison_data)
print("\n" + comparison_df.to_string(index=False))

# Calculate improvements
print("\n" + "="*70)
print("Improvement Analysis")
print("="*70)

loss_improvement = ((base_loss - sft_loss) / base_loss) * 100 if base_loss > 0 else 0
acc_improvement = ((sft_accuracy - base_accuracy) / base_accuracy) * 100 if base_accuracy > 0 else 0
balanced_acc_improvement = ((sft_balanced_accuracy - base_balanced_accuracy) / base_balanced_accuracy) * 100 if base_balanced_accuracy > 0 else 0
macro_f1_improvement = ((sft_macro_f1 - base_macro_f1) / base_macro_f1) * 100 if base_macro_f1 > 0 else 0

print(f"\nLoss Improvement: {loss_improvement:+.2f}% ({'✓ Better' if loss_improvement > 0 else '✗ Worse'})")
print(f"Accuracy Improvement: {acc_improvement:+.2f}% ({'✓ Better' if acc_improvement > 0 else '✗ Worse'})")
print(f"Balanced Accuracy Improvement: {balanced_acc_improvement:+.2f}% ({'✓ Better' if balanced_acc_improvement > 0 else '✗ Worse'})")
print(f"Macro F1 Improvement: {macro_f1_improvement:+.2f}% ({'✓ Better' if macro_f1_improvement > 0 else '✗ Worse'})")

# Step 5: Per-class comparison
print("\n" + "="*70)
print("Step 5: Per-Class Performance Comparison")
print("="*70)

print("\nBase Model Classification Report:")
print(classification_report(base_true_labels, base_predictions, 
                          target_names=[str(cls) for cls in le.classes_], 
                          zero_division=0))

print("\nFine-Tuned Model Classification Report:")
if 'sft_predictions' in locals():
    print(classification_report(sft_true_labels, sft_predictions, 
                              target_names=[str(cls) for cls in le.classes_], 
                              zero_division=0))
else:
    print("(Using training metrics - detailed per-class report not available)")

# Step 6: Prediction distribution comparison
print("\n" + "="*70)
print("Step 6: Prediction Distribution Comparison")
print("="*70)

from collections import Counter

base_pred_dist = Counter(base_predictions)
sft_pred_dist = Counter(sft_predictions) if 'sft_predictions' in locals() else Counter()

print("\nBase Model Prediction Distribution:")
for label_id, count in sorted(base_pred_dist.items()):
    class_name = le.inverse_transform([label_id])[0]
    percentage = (count / len(base_predictions)) * 100
    print(f"  Class {label_id} ('{class_name}'): {count} predictions ({percentage:.1f}%)")

if sft_pred_dist:
    print("\nFine-Tuned Model Prediction Distribution:")
    for label_id, count in sorted(sft_pred_dist.items()):
        class_name = le.inverse_transform([label_id])[0]
        percentage = (count / len(sft_predictions)) * 100
        print(f"  Class {label_id} ('{class_name}'): {count} predictions ({percentage:.1f}%)")

print("\n" + "="*70)
print("Comparison Complete!")
print("="*70)


Comparison: Base Model vs Fine-Tuned Model

Step 1: Loading Base BERT Model (No Fine-Tuning)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Base model loaded on device: cpu

Step 2: Evaluating Base Model

Base Model Results:
  Loss: 1.2982
  Accuracy: 0.0053 (0.53%)
  Balanced Accuracy: 0.3333 (33.33%)
  Micro F1: 0.0053
  Weighted F1: 0.0001
  Macro F1: 0.0035

Step 3: Getting Fine-Tuned Model Results



Fine-Tuned Model Results:
  Loss: 0.9570
  Accuracy: 0.8138 (81.38%)
  Balanced Accuracy: 0.3333 (33.33%)
  Micro F1: 0.8138
  Weighted F1: 0.7303
  Macro F1: 0.2991

Step 4: Side-by-Side Comparison

                Metric Base Model (No SFT) Fine-Tuned Model (SFT)
Loss (Lower is Better)              1.2982                 0.9570
              Accuracy      0.0053 (0.53%)        0.8138 (81.38%)
     Balanced Accuracy     0.3333 (33.33%)        0.3333 (33.33%)
              Micro F1              0.0053                 0.8138
           Weighted F1              0.0001                 0.7303
              Macro F1              0.0035                 0.2991

Improvement Analysis

Loss Improvement: +26.28% (✓ Better)
Accuracy Improvement: +15200.00% (✓ Better)
Balanced Accuracy Improvement: +0.00% (✗ Worse)
Macro F1 Improvement: +8335.19% (✓ Better)

Step 5: Per-Class Performance Comparison

Base Model Classification Report:
              precision    recall  f1-score   support

          